### Verify and Visualize Labeled Data

In [2]:
import pandas as pd

# Load the data
position_data = pd.read_csv('../data-collection/2024-12-10_22-37-46_Data/position_data.csv')

# Define thresholds
speed_threshold = 0.1  # Consider speed <= 0.1 as "stop"
turn_threshold = 10    # Degrees to detect turns (adjust as needed)

# Calculate course change
position_data['course_diff'] = position_data['course'].diff().fillna(0)

# Assign labels
def label_row(row):
    if row['speed'] <= speed_threshold:
        return 'Stop'
    elif abs(row['course_diff']) > turn_threshold:
        return 'Turn Left' if row['course_diff'] < 0 else 'Turn Right'
    else:
        return 'Moving Forward'

position_data['Label'] = position_data.apply(label_row, axis=1)

# Define the save path in the data-collection folder
output_file = '../data-collection/2024-12-10_22-37-46_Data/labeled_position_data.csv'

# Save the labeled data
position_data.to_csv(output_file, index=False)
print(f"Labeled data saved as {output_file}")

Labeled data saved as ../data-collection/2024-12-10_22-37-46_Data/labeled_position_data.csv


In [4]:
from folium import Map, CircleMarker
from folium.plugins import MarkerCluster
from branca.element import Template, MacroElement

# Define the legend as an HTML template
legend_html = '''
<div style="
    position: fixed; 
    bottom: 50px; left: 50px; width: 200px; height: 150px; 
    background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
    ">
    &nbsp; <b>Legend</b> <br>
    &nbsp; <i style="color: red;">●</i> Stop <br>
    &nbsp; <i style="color: blue;">●</i> Moving Forward <br>
    &nbsp; <i style="color: green;">●</i> Turn Left <br>
    &nbsp; <i style="color: orange;">●</i> Turn Right <br>
</div>
'''

# Create a folium map
map_center = [position_data['latitude'][0], position_data['longitude'][0]]
labeled_map = folium.Map(location=map_center, zoom_start=15)

# Add points with colors based on labels
for _, row in position_data.iterrows():
    label_color = {
        'Stop': 'red',
        'Moving Forward': 'blue',
        'Turn Left': 'green',
        'Turn Right': 'orange'
    }[row['Label']]
    CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        color=label_color,
        fill=True,
        fill_color=label_color
    ).add_to(labeled_map)

# Add the legend to the map
labeled_map.get_root().html.add_child(folium.Element(legend_html))

# Save the map
labeled_map.save('labeled_gps_map_with_legend.html')
print("Labeled map with legend saved as labeled_gps_map_with_legend.html")


Labeled map with legend saved as labeled_gps_map_with_legend.html


## Apply K-Means Clustering for GPS Data Labeling

### Experimental: K-Means Clustering for GPS Labeling
This section explores using K-Means clustering for labeling GPS data. While the logic-based method provided more accurate results, K-Means was attempted to automate the process. Due to the simplicity of the dataset and lack of distinct clustering features, K-Means did not perform as expected. The code is retained for reference and future experimentation.


In [4]:
import pandas as pd
from sklearn.cluster import KMeans

# Load GPS data
position_data = pd.read_csv('../data-collection/2024-12-10_22-37-46_Data/position_data.csv')

# Select features for clustering
features = position_data[['speed', 'course']].fillna(0)  # Replace NaNs with 0 if any

# Apply K-Means
n_clusters = 4  # Number of movement types (e.g., Stop, Moving Forward, Turning)
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
position_data['Cluster'] = kmeans.fit_predict(features)

# Analyze the clusters
print("Cluster Centers:")
print(kmeans.cluster_centers_)

# Assign human-readable labels to clusters
# (Modify the logic based on your cluster centers)
def assign_label(cluster):
    if cluster == 0:  # Replace with your logic
        return 'Stop'
    elif cluster == 1:
        return 'Moving Forward'
    elif cluster == 2:
        return 'Turn Left'
    else:
        return 'Turn Right'

position_data['Label'] = position_data['Cluster'].apply(assign_label)

# Save labeled data
position_data.to_csv('../data-collection/2024-12-10_22-37-46_Data/kmeans_labeled_data.csv', index=False)
print("Labeled data saved as kmeans_labeled_data.csv")


Cluster Centers:
[[3.35142105e+00 5.69899825e+01]
 [4.25122892e+00 1.59876434e+02]
 [1.01645833e-01 1.03831250e+00]
 [4.22314706e+00 9.90800126e+01]]
Labeled data saved as kmeans_labeled_data.csv


In [5]:
import folium  # Import folium for map visualization
from folium import Map, CircleMarker
from folium.plugins import MarkerCluster
from branca.element import Template, MacroElement

# Define the legend as an HTML template
legend_html = '''
<div style="
    position: fixed; 
    bottom: 50px; left: 50px; width: 200px; height: 150px; 
    background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
    ">
    &nbsp; <b>Legend</b> <br>
    &nbsp; <i style="color: red;">●</i> Stop <br>
    &nbsp; <i style="color: blue;">●</i> Moving Forward <br>
    &nbsp; <i style="color: green;">●</i> Turn Left <br>
    &nbsp; <i style="color: orange;">●</i> Turn Right <br>
</div>
'''

# Create a folium map
map_center = [position_data['latitude'][0], position_data['longitude'][0]]
labeled_map = folium.Map(location=map_center, zoom_start=15)

# Add points with colors based on labels
for _, row in position_data.iterrows():
    label_color = {
        'Stop': 'red',
        'Moving Forward': 'blue',
        'Turn Left': 'green',
        'Turn Right': 'orange'
    }[row['Label']]
    CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        color=label_color,
        fill=True,
        fill_color=label_color
    ).add_to(labeled_map)

# Add the legend to the map
labeled_map.get_root().html.add_child(folium.Element(legend_html))

# Save the map
labeled_map.save('labeled_gps_map_with_legend_Kmeans.html')
print("Labeled map with legend saved as labeled_gps_map_with_legend_Kmeans.html")


Labeled map with legend saved as labeled_gps_map_with_legend_Kmeans.html
